Import Dependencies

In [1]:
import numpy as np
import pandas as pd

Ignore any warning messages

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import sqlite3 as sq
import sqlalchemy
from sqlalchemy import create_engine, func
from sqlalchemy.orm import Session
from sqlalchemy.ext.automap import automap_base

Load data into dataframe

In [4]:
# Load the .csv dataset.
file_path = "../Resources/Use me 2020 NFL 3rd & Long Raw Dataset.csv"
df = pd.read_csv(file_path)
df 

,GameId,GameDate,Quarter,Minute,Second,OffenseTeam,DefenseTeam,Down,ToGo,Yards,...,IsTwoPointConversion,IsTwoPointConversionSuccessful,RushDirection,YardLineFixed,YardLineDirection,IsPenaltyAccepted,PenaltyTeam,IsNoPlay,PenaltyType,PenaltyYards
0,2020122602,12/26/2020,2,9,44,LV,MIA,2,10,-9,...,0,0,NaN,18,OPP,0,NaN,0,NaN,0
1,2020122602,12/26/2020,2,9,3,LV,MIA,3,19,18,...,0,0,LEFT GUARD,27,OPP,0,NaN,0,NaN,0
2,2020122500,12/25/2020,3,1,17,MIN,NO,3,9,0,...,0,0,NaN,33,OPP,0,NO,1,DEFENSIVE HOLDING,0
3,2020122500,12/25/2020,3,1,12,MIN,NO,1,10,0,...,0,0,NaN,18,OPP,1,NO,1,DEFENSIVE PASS INTERFERENCE,17
4,2020122500,12/25/2020,3,1,6,MIN,NO,1,1,-3,...,0,0,LEFT GUARD,1,OPP,0,NaN,0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46184,2020091300,9/13/2020,4,1,38,ATL,SEA,2,9,2,...,0,0,NaN,20,OPP,0,NaN,0,NaN,0
46185,2020091300,9/13/2020,4,5,18,NaN,NaN,0,0,0,...,0,0,NaN,0,OPP,0,NaN,0,NaN,0
46186,2020091300,9/13/2020,4,2,0,ATL,SEA,1,10,1,...,0,0,NaN,21,OPP,0,NaN,0,NaN,0
46187,2020091300,9/13/2020,4,2,0,NaN,NaN,0,0,0,...,0,0,NaN,0,OPP,0,NaN,0,NaN,0


Set to display mximum # of columns

In [5]:
pd.set_option("display.max_columns",None)

Get count of null values

In [6]:
df.isna().sum()

GameId                                0
GameDate                              0
Quarter                               0
Minute                                0
Second                                0
OffenseTeam                        3955
DefenseTeam                        3955
Down                                  0
ToGo                                  0
Yards                                 0
Yards (+,-)                           0
Successful 3rd Down Y/N               0
YardLine                              0
SeriesFirstDown                       0
NextScore                             0
Description                           0
TeamWin                               0
SeasonYear                            0
Formation                          1034
PlayType                           2131
IsRush                                0
IsPass                                0
IsIncomplete                          0
IsTouchdown                           0
PassType                          27327


Set framework for output dataframe
Fill blank cells with unk

In [7]:
df[["OffenseTeam","DefenseTeam","Formation", "PlayType", "PassType",
             "Challenger", "RushDirection", "PenaltyTaeam","PenaltyType"]] = \
df[["OffenseTeam","DefenseTeam","Formation", "PlayType", "PassType",
             "Challenger", "RushDirection", "PenaltyTeam","PenaltyType"]].fillna("unk")

Get count of blank cells
Count now zero

In [8]:
df.isna().sum()

GameId                            0
GameDate                          0
Quarter                           0
Minute                            0
Second                            0
OffenseTeam                       0
DefenseTeam                       0
Down                              0
ToGo                              0
Yards                             0
Yards (+,-)                       0
Successful 3rd Down Y/N           0
YardLine                          0
SeriesFirstDown                   0
NextScore                         0
Description                       0
TeamWin                           0
SeasonYear                        0
Formation                         0
PlayType                          0
IsRush                            0
IsPass                            0
IsIncomplete                      0
IsTouchdown                       0
PassType                          0
IsSack                            0
IsChallenge                       0
IsChallengeReversed         

Push dataframe to sqlite

In [13]:
sql_data = 'df.sqlite'

In [14]:
# Create connection & push the data:
conn = sq.connect(sql_data)
cur = conn.cursor()

Run initially to set up the sqlite tables
Do not run after intial setup

In [15]:
cur.executescript('''

DROP TABLE IF EXISTS "THIRD_LONG";
CREATE TABLE "THIRD_LONG" (
    "index" INTEGER PRIMARY KEY AUTOINCREMENT,
    "Quarter" INTEGER NOT NULL,
    "Down" INTEGER NOT NULL,
    "ToGo" INTEGER NOT NULL,
    "Successful 3rd Down Y/N" CHAR NOT NULL,
    "Formation" CHAR NOT NULL,
    "PlayType" CHAR NOT NULL,
    "GameId" INTEGER NOT NULL,
    "GameDate" Date NOT NULL,
    "Minute" CHAR NOT NULL,
    "Second" CHAR NOT NULL,
    "OffenseTeam" CHAR NOT NULL,
    "DefenseTeam" CHAR NOT NULL,
    "Yards" INTEGER NOT NULL,
    "Yards (+,-)" INTEGER NOT NULL,
    "YardLine" INTEGER NOT NULL,
    "SeriesFirstDown"INTEGER NOT NULL,
    "NextScore" INTEGER NOT NULL,
    "Description" CHAR NOT NULL,
    "TeamWin" CHAR NOT NULL,
    "SeasonYear" INTEGER NOT NULL,
    "IsRush" CHAR NOT NULL,
    "IsPass" INTEGER NOT NULL,
    "IsIncomplete"INTEGER NOT NULL,
    "IsTouchdown" INTEGER NOT NULL,
    "PassType"CHAR NOT NULL,
    "IsSack" INTEGER NOT NULL,
    "IsChallenge" INTEGER NOT NULL,
    "IsChallengeReversed"INTEGER NOT NULL,
    "Challenger" CHAR NOT NULL,
    "IsMeasurement" INTEGER NOT NULL,
    "IsInterception" INTEGER NOT NULL,
    "IsFumble" INTEGER NOT NULL,
    "IsPenalty" INTEGER NOT NULL,
    "IsTwoPointConversion" INTEGER NOT NULL,
    "IsTwoPointConversionSuccessful" INTEGER NOT NULL,
    "RushDirection" CHAR NOT NULL,
    "YardLineFixed" INTEGER NOT NULL,
    "YardLineDirection" CHAR NOT NULL,
    "IsPenaltyAccepted" INTEGER NOT NULL,
    "PenaltyTeam" CHAR NOT NULL,
    "IsNoPlay" INTEGER NOT NULL,
    "PenaltyType" CHAR NOT NULL,
    "PenaltyYards" INTEGER NOT NULL
 
    
    );

''')

Append THE NEW DATA (THIRD_LONG) TO THE EXISTING SQLITE TABLE:

In [16]:
df.to_sql("THIRD_LONG", conn, if_exists='append', index=True)
conn.commit()
conn.close()

In [17]:
engine = create_engine("sqlite:///df.sqlite")

# REFLECT AN EXISTING DATABASE INTO A NEW MODEL:
Base = automap_base()

# REFLECT THE TABLES:
Base.prepare(engine, reflect=True)

# SAVE REFERENCE TO THE COMBINED TABLE:
THIRD_LONG = Base.classes.THIRD_LONG

In [18]:
# READ SQLITE QUERY RESULTS INTO A PANDAS DATAFRAME:
con = sq.connect("df.sqlite")

In [19]:
# IMPORT THIRD LONG DATA:
NFL_20 = pd.read_sql_query("\
SELECT * FROM THIRD_LONG", con).drop('index',1)
NFL_20

,Quarter,Down,ToGo,Successful 3rd Down Y/N,Formation,PlayType,GameId,GameDate,Minute,Second,OffenseTeam,DefenseTeam,Yards,"Yards (+,-)",YardLine,SeriesFirstDown,NextScore,Description,TeamWin,SeasonYear,IsRush,IsPass,IsIncomplete,IsTouchdown,PassType,IsSack,IsChallenge,IsChallengeReversed,Challenger,IsMeasurement,IsInterception,IsFumble,IsPenalty,IsTwoPointConversion,IsTwoPointConversionSuccessful,RushDirection,YardLineFixed,YardLineDirection,IsPenaltyAccepted,PenaltyTeam,IsNoPlay,PenaltyType,PenaltyYards
0,2,2,10,4th Down,SHOTGUN,SACK,2020122602,12/26/2020,9,44,LV,MIA,-9,-19,82,0,0,(9:44) (SHOTGUN) 4-D.CARR SACKED AT MIA 27 FOR...,0,2020,0,0,0,0,unk,1,0,0,unk,0,0,0,0,0,0,unk,18,OPP,0,unk,0,unk,0
1,2,3,19,4th Down,SHOTGUN,RUSH,2020122602,12/26/2020,9,3,LV,MIA,18,-1,73,0,0,(9:03) (SHOTGUN) 30-J.RICHARD LEFT GUARD PUSHE...,0,2020,1,0,0,0,unk,0,0,0,unk,0,0,0,0,0,0,LEFT GUARD,27,OPP,0,unk,0,unk,0
2,3,3,9,4th Down,SHOTGUN,PASS,2020122500,12/25/2020,1,17,MIN,NO,0,-9,67,0,0,(1:17) (SHOTGUN) 8-K.COUSINS PASS INCOMPLETE S...,0,2020,0,1,1,0,SHORT LEFT,0,0,0,unk,0,0,0,1,0,0,unk,33,OPP,0,NO,1,DEFENSIVE HOLDING,0
3,3,1,10,4th Down,UNDER CENTER,PASS,2020122500,12/25/2020,1,12,MIN,NO,0,-10,82,0,0,(1:12) 8-K.COUSINS PASS INCOMPLETE DEEP LEFT T...,0,2020,0,1,1,0,DEEP LEFT,0,0,0,unk,0,0,0,1,0,0,unk,18,OPP,1,NO,1,DEFENSIVE PASS INTERFERENCE,17
4,3,1,1,4th Down,UNDER CENTER,RUSH,2020122500,12/25/2020,1,6,MIN,NO,-3,-4,99,0,0,(1:06) 69-R.HILL REPORTED IN AS ELIGIBLE. 33-...,0,2020,1,0,0,0,unk,0,0,0,unk,0,0,0,0,0,0,LEFT GUARD,1,OPP,0,unk,0,unk,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46184,4,2,9,4th Down,NO HUDDLE SHOTGUN,PASS,2020091300,9/13/2020,1,38,ATL,SEA,2,-7,80,0,0,"(1:38) (NO HUDDLE, SHOTGUN) 2-M.RYAN PASS SHOR...",0,2020,0,1,0,0,SHORT MIDDLE,0,0,0,unk,0,0,0,0,0,0,unk,20,OPP,0,unk,0,unk,0
46185,4,0,0,1st Down,UNDER CENTER,TIMEOUT,2020091300,9/13/2020,5,18,unk,unk,0,0,100,1,0,TIMEOUT #3 BY ATL AT 05:18.,0,2020,0,0,0,0,unk,0,0,0,unk,0,0,0,0,0,0,unk,0,OPP,0,unk,0,unk,0
46186,4,1,10,4th Down,SHOTGUN,PASS,2020091300,9/13/2020,2,0,ATL,SEA,1,-9,79,0,0,(2:00) (SHOTGUN) 2-M.RYAN PASS SHORT RIGHT TO ...,0,2020,0,1,0,0,SHORT RIGHT,0,0,0,unk,0,0,0,0,0,0,unk,21,OPP,0,unk,0,unk,0
46187,4,0,0,1st Down,UNDER CENTER,unk,2020091300,9/13/2020,2,0,unk,unk,0,0,100,1,0,TWO-MINUTE WARNING,0,2020,0,0,0,0,unk,0,0,0,unk,0,0,0,0,0,0,unk,0,OPP,0,unk,0,unk,0


In [20]:
con.close()